   Copyright 2019 Daniel Kowatsch

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.


# Differential Privacy and Memorization

This notebook provides an example of the memorization effect occurring in recurrent neural networks and how to train a model with Tensorflow's differentially private optimizers in order to limit or prevent memorization.
For this purpose we train a character-level language model, once with regular Adam, once with differentially private Adam. We use an estimate of the z-score of the sequence probability distribution as a measure of the memorization and compare the results. More details will be explained later on.


## Requirements and Imports

Before we can start, you should ensure that you have a valid iPython kernel.
This notebook was implemented using Python 3.7, older versions might work but are not officially supported.
The kernel should also have the following packages installed:
1. numpy
2. matplotlib
3. tensorflow (tested with 1.13)
4. tensorflow privacy (https://github.com/tensorflow/privacy, commit 51e29667d97879b1f0adba940eceaa24e9266b1f)

For Tensorflow Privacy please follow the installation guide in the git.

Alternatively, you can try the `setup.sh` script in this git. You may have to restart the jupyter notebook in order to find the new kernel.

If you have installed everything, the following cell should run through.

In [ ]:
# Required imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import os
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import tensorflow as tf
from privacy.analysis import privacy_ledger
from privacy.analysis.rdp_accountant import compute_rdp_from_ledger
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers import dp_optimizer

## Global Variables for Configuration

The `noise_multiplier` will influence how strongly noise is applied in the differentially private optimization. More noise results in worse performance but better privacy.

A larger number `microbatches` can also positively influence privacy but results in higher ressource consumption and might lead to memory issues.

In order to evaluate memorization we will later add a constructed secret in the data set. `secret_format` describes how the secret will look like. `{}` will be filled by random digits, `_` represents blank spaces while blank spaces are used for seperating characters in the sequence.

In [ ]:
# Define global variables
# Learning rate for training
learning_rate = .001
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
l2_norm_clip = 1.0
# Batch size
batch_size = 16
# Seed used in random operations
seed = 42
# Number of epochs
epochs = 10
# Number of microbatches (must evenly divide batch_size)
microbatches = 16
# Model directory
model_dir_regular = '../model_regular_10'
model_dir_dp = '../model_dp_10'
# Directory containing the PTB data.
data_dir = '../data/pennchar'
# Format of the secret injected in the data set.
secret_format = 'm y _ c r e d i t _ c a r d _ n u m b e r _ i s _ {} {} {} {} {} {} {} {} {}'
# If True, load the latest checkpoint from model_dir. If False, train from scratch.
load_model = False

In [ ]:
# Here we set the logging level of tensorflow and store some variables regarding the data set for later use.
tf.logging.set_verbosity(tf.logging.WARN)
if batch_size % microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

SEQ_LEN = 20
NB_TRAIN = 4975360
EPSILON_LIST = []
Z_SCORE_LIST = []

## Data Set

As previously mentioned, we use the Penn Treebank character data set. A copy of this is already included in this git repository and can be found in the data directory. If you are unable to load the data set, please ensure `data_dir` points to the correct directory.  

The following method is a helper to load the data set and randomly insert the secret. Thus, you should make a safety copy of the data set.

In [ ]:
# Define a method for loading and modifying the data set
def load_data(data_dir, secret_format, seed):
    """Load training and validation data."""
    assert os.path.exists(data_dir), 'The data set can not be found at {}.'.format(os.path.abspath(data_dir)) + \
                                     'Please ensure you have downloaded the data set and specified the correct path.' 
    pickled_data_path = os.path.join(data_dir, 'corpus.{}.data'.format(hashlib.md5('{}{}'.format(secret_format, seed).encode()).hexdigest()))
    if os.path.isfile(pickled_data_path):
        dataset = pickle.load(open(pickled_data_path, 'rb'))
    else:
        # Set seed for reproducibility
        if seed is not None:
            random.seed(seed)

        # Generate the secret
        secret_plain = secret_format.format(*(random.sample(range(0, 10), 9)))
        print('secret:', secret_plain)

        # Create paths for later use
        train_file_path = os.path.join(data_dir, 'train.txt')
        test_file_path = os.path.join(data_dir, 'test.txt')
        train_file_path_secret_injected = os.path.join(data_dir, '{}_train.txt'.format(secret_plain)).replace(' ', '')

        # Insert secret in dataset
        with open(train_file_path, 'r') as f:
            contents = f.readlines()
            index = random.randint(0, len(contents))
            contents.insert(index, ' ' + secret_plain + ' \n')

        # Store dataset with injected secret in other file
        with open(train_file_path_secret_injected, 'w') as f:
            contents = ''.join(contents)
            f.write(contents)

        # Extract stuff for using dataset for training
        train_txt = open(train_file_path_secret_injected).read().split()
        test_txt = open(test_file_path).read().split()
        keys = sorted(set(train_txt))
        remap = {k: i for i, k in enumerate(keys)}
        train_data = np.array([remap[character] for character in train_txt], dtype=np.uint8)
        test_data = np.array([remap[character] for character in test_txt], dtype=np.uint8)
        secret_sequence = np.array([remap[character] for character in secret_plain.split()])
        dataset = {'train': train_data, 'test': test_data, 'num_classes': len(keys), 'dictionary': remap, 'seed': seed,
                   'secret_plain': secret_plain, 'secret_format': secret_format, 'secret_sequence': secret_sequence}
        pickle.dump(dataset, open(pickled_data_path, 'wb'))

    return dataset

In [ ]:
# Load training and test data.
dataset = load_data(data_dir=data_dir, secret_format=secret_format, seed=seed)

train_data = dataset['train']
test_data = dataset['test']

secret_sequence = dataset['secret_sequence']

After we have loaded the data set in the last cell, we will now define some functions in order to feed the data set to the TensorFlow estimators we will use later on. The calculations beforehand are to ensure we don't get problems with a number of data points which is not divisable by the batch length. 

In [ ]:
# Create tf.Estimator input functions for the training and test data.
batch_len = batch_size * SEQ_LEN
# Calculate remainders
remainder_train = len(train_data) % batch_len
remainder_test = len(test_data) % batch_len
# In case batch_len divides the number of characters in the dataset, the wouldn't have labels for the last entry
if remainder_train != 0:
    train_data_end = len(train_data) - remainder_train
else:
    train_data_end = len(train_data) - batch_len
train_label_end = train_data_end + 1
# Set the number of training data accordingly, calling the estimator beforehand might cause problems
NB_TRAIN = train_data_end
# Same for the test data
if remainder_test != 0:
    test_data_end = len(test_data) - remainder_test
else:
    test_data_end = len(test_data) - batch_len
test_label_end = test_data_end + 1
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': train_data[:train_data_end].reshape((-1, SEQ_LEN))},
    y=train_data[1:train_label_end].reshape((-1, SEQ_LEN)),
    batch_size=batch_len,
    num_epochs=epochs,
    shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test_data[:test_data_end].reshape((-1, SEQ_LEN))},
    y=test_data[1:test_label_end].reshape((-1, SEQ_LEN)),
    batch_size=batch_len,
    num_epochs=1,
    shuffle=False)

## Training a Model with regular Optimization

In order to show, that memorization occurs, we will first train a simple 2-layer LSTM model and plot the estimated z-score. We will deactivate differentially private optimization for now, but we will define the network to allow differentially private optimization to make it more obvious that we are just changing the optimization algorithm.

In [ ]:
dpsgd = False

First, we define a training hook to print epsilon values of the differentially private Adam after each epoch. This will not be important for now but used later when we train with differentially private Adam.  

In [ ]:
# We define a training hook in order to be able to periodically print the epsilon values
class EpsilonPrintingTrainingHook(tf.estimator.SessionRunHook):
    """Training hook to print current value of epsilon after an epoch."""

    def __init__(self, ledger):
        """Initalizes the EpsilonPrintingTrainingHook.

        Args:
          ledger: The privacy ledger.
        """
        self._samples, self._queries = ledger.get_unformatted_ledger()

    def end(self, session):
        orders = [1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64))
        samples = session.run(self._samples)
        queries = session.run(self._queries)
        formatted_ledger = privacy_ledger.format_ledger(samples, queries)
        rdp = compute_rdp_from_ledger(formatted_ledger, orders)
        eps = get_privacy_spent(orders, rdp, target_delta=1e-7)[0]
        EPSILON_LIST.append(eps)
        print('For delta=1e-7, the current epsilon is: %.2f' % eps)

Now we can define the model. 

In [ ]:
# Define the model using Tensorflow estimators
def rnn_model_fn(features, labels, mode):  # pylint: disable=unused-argument
    """Model function for a RNN."""

    # Define RNN architecture using tf.keras.layers.
    x = features['x']
    input_layer = x[:]
    input_one_hot = tf.one_hot(input_layer, 200)
    lstm = tf.keras.layers.LSTM(200, return_sequences=True).apply(input_one_hot)
    lstm = tf.keras.layers.LSTM(200, return_sequences=True).apply(lstm)
    logits = tf.keras.layers.Dense(50).apply(lstm)

    if mode != tf.estimator.ModeKeys.PREDICT:
        # Calculate loss as a vector (to support microbatches in DP-SGD).
        vector_loss = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.cast(tf.one_hot(labels, 50), dtype=tf.float32),
            logits=logits)
        # Define mean of loss across minibatch (for reporting through tf.Estimator).
        scalar_loss = tf.reduce_mean(vector_loss)

    # Configure the training op (for TRAIN mode).
    if mode == tf.estimator.ModeKeys.TRAIN:
        if dpsgd:
            ledger = privacy_ledger.PrivacyLedger(
                population_size=NB_TRAIN,
                selection_probability=(batch_size*SEQ_LEN / NB_TRAIN),
                max_samples=1e6,
                max_queries=1e6)
            optimizer = dp_optimizer.DPAdamGaussianOptimizer(
                l2_norm_clip=l2_norm_clip,
                noise_multiplier=noise_multiplier,
                num_microbatches=microbatches,
                learning_rate=learning_rate,
                unroll_microbatches=True,
                ledger=ledger)
            training_hooks = [
                EpsilonPrintingTrainingHook(ledger)
            ]
            opt_loss = vector_loss
        else:
            optimizer = tf.train.AdamOptimizer(
                learning_rate=learning_rate)
            training_hooks = []
            opt_loss = scalar_loss
        global_step = tf.train.get_global_step()
        train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          train_op=train_op,
                                          training_hooks=training_hooks)

    # Add evaluation metrics (for EVAL mode).
    elif mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy':
                tf.metrics.accuracy(
                    labels=tf.cast(labels, dtype=tf.int32),
                    predictions=tf.argmax(input=logits, axis=2))
        }
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          eval_metric_ops=eval_metric_ops)
    elif mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=tf.nn.softmax(logits=logits))


In [ ]:
warm_start_from = {'warm_start_from': model_dir_regular} if load_model else {}

# Instantiate the tf.Estimator.
conf = tf.estimator.RunConfig(save_summary_steps=1000)
lm_classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,
                                        model_dir=model_dir_regular,
                                       config=conf,
                                       **warm_start_from)

Next, we will define methods to estimate the memorization effect. For this we use the log-perplexity as a basis.

Note: For simplicity, we ignored the influence of the first element on the log-perplexity. For our purpose this is irrelevant since, in our example, the first element of the sequence is fixed and therefore constitutes a constant offset.

In [ ]:
# Define a function to calculate the log-perplexity of a secret (at least approximately).
def log_perplexity(estimator, sequence):
    assert 0 < len(sequence.shape) <= 2, "Length of the shape of the sequence has to be 1 or 2, currently it is {}".\
        format(len(sequence.shape))
    if len(sequence.shape) == 1:
        formatted_sequence = sequence.reshape((1, -1))
    else:
        formatted_sequence = sequence
    sequence_input = tf.estimator.inputs.numpy_input_fn(
        x={'x': formatted_sequence},
        batch_size=20,
        num_epochs=1,
        shuffle=False)
    sequence_length = formatted_sequence.shape[1]
    prediction_generator = estimator.predict(sequence_input)
    log_perplexity_list = []
    for i, prediction in enumerate(prediction_generator):
        sequence_probabilities = prediction[(range(sequence_length-1), formatted_sequence[i, 1:])]
        negative_log_probability = np.sum(-np.log(sequence_probabilities))
        log_perplexity_list.append(negative_log_probability)
    return log_perplexity_list

Now, we can estimate the z-score. In order to do this, we randomly sample 1,000 potential secrets and calculate their log-perplexities. These are approximately normal distributed. So we transform these and the log-perplexity of the actual secret to a standard normal distribution. Because of this, a low z-score of the secret corresponds to the secret being very probable under the model, indicating it was contained in the data set. 

In [ ]:
# Function for estimating the z-score
def estimate_z_score(estimator, secret, secret_format, dictionary, seed=42, sample_size=1000):
    secret_log_perplexity = log_perplexity(estimator=estimator, sequence=secret)
    np.random.seed(seed=seed)
    samples_of_random_space = np.random.randint(0, 10, (sample_size, 9))
    list_of_samples = []
    for i in range(sample_size):
        sample = secret_format.format(*samples_of_random_space[i]).split()
        int_representation = [dictionary[character] for character in sample]
        list_of_samples.append(int_representation)
    sample_log_perplexity_list = log_perplexity(estimator, np.array(list_of_samples))
    mean = np.mean(sample_log_perplexity_list)
    std = np.std(sample_log_perplexity_list)
    z_score = (secret_log_perplexity - mean)/std
    return z_score

Next, we can finally train our model on the modified data set. We also print the z-scores after each epoch.

In [ ]:
# Training loop.
steps_per_epoch = NB_TRAIN // batch_len
for epoch in range(1, epochs + 1):
    print('epoch', epoch)
    # Train the model for one epoch.
    lm_classifier.train(input_fn=train_input_fn, steps=steps_per_epoch)

    if epoch % 5 == 1:
        name_input_fn = [('Train', train_input_fn), ('Eval', eval_input_fn)]
        for name, input_fn in name_input_fn:
            # Evaluate the model and print results
            eval_results = lm_classifier.evaluate(input_fn=input_fn)
            result_tuple = (epoch, eval_results['accuracy'], eval_results['loss'])
            print(name, 'accuracy after %d epochs is: %.3f (%.4f)' % result_tuple)

    z_score = estimate_z_score(estimator=lm_classifier,
                                secret=secret_sequence,
                                secret_format=dataset['secret_format'],
                                dictionary=dataset['dictionary'],
                                seed=seed + 1,
                                sample_size=1000)
    Z_SCORE_LIST.append(z_score)
    print("z-score: {}".format(z_score))        

In [ ]:
# Quickly save the z-scores for later use
np.save('regular_z_scores.npy', np.array(Z_SCORE_LIST))

In order to visualize the memorization effect, we plot the z-scores.

In [ ]:
# Plotting z-scores
x = range(1, epoch + 1)
plt.plot(x, Z_SCORE_LIST, label='z-score')
plt.xlabel('Epoch')
plt.ylabel('z-score')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.savefig("z_score_{}_regular.png".format(dataset['secret_format']).replace(' ', ''))
plt.show()
plt.close()
print(Z_SCORE_LIST)

The distribution of the log-perplexity of potential secrets is approximiately a normal distribution.
For visualization we plot a normal distribution and show where the current secret is placed, given the models log-perplexity for the secret. Here, the further to the left of the plot, the more probable the sequence is under the model.

In [ ]:
x = np.linspace(-5, 5, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1), label='Normal distribution')
plt.scatter(Z_SCORE_LIST, stats.norm.pdf(Z_SCORE_LIST), marker='x', color='red', label='Secret Probabilities')
plt.xlabel('Standard deviations from mean')
plt.ylabel('Probability')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.show()

In the last plots we have seen that the secret is probable under the model and an attacker can assume that the training data contains the secret.

## Training a Model with Differentially Private Optimization

For comparison, we will also train a model with differentially private optimization. This is noticeably slower and might take a while.

In [ ]:
dpsgd = True

In [ ]:
warm_start_from = {'warm_start_from': model_dir_dp} if load_model else {}

# Instantiate the tf.Estimator.
conf = tf.estimator.RunConfig(save_summary_steps=1000)
lm_classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,
                                        model_dir=model_dir_dp,
                                       config=conf,
                                       **warm_start_from)

In [ ]:
# Training loop.
steps_per_epoch = NB_TRAIN // batch_len
Z_SCORE_LIST = []
EPSILON_LIST = []
for epoch in range(1, epochs + 1):
    print('epoch', epoch)
    # Train the model for one epoch.
    lm_classifier.train(input_fn=train_input_fn, steps=steps_per_epoch)

    if epoch % 5 == 1:
        name_input_fn = [('Train', train_input_fn), ('Eval', eval_input_fn)]
        for name, input_fn in name_input_fn:
            # Evaluate the model and print results
            eval_results = lm_classifier.evaluate(input_fn=input_fn)
            result_tuple = (epoch, eval_results['accuracy'], eval_results['loss'])
            print(name, 'accuracy after %d epochs is: %.3f (%.4f)' % result_tuple)

    z_score = estimate_z_score(estimator=lm_classifier,
                                secret=secret_sequence,
                                secret_format=dataset['secret_format'],
                                dictionary=dataset['dictionary'],
                                seed=seed + 1,
                                sample_size=1000)
    Z_SCORE_LIST.append(z_score)
    print("z-score: {}".format(z_score))        

In [ ]:
# Quickly save the z-scores and epsilon values for later use
np.save('dp_z_scores.npy', np.array(Z_SCORE_LIST))
np.save('epsilon.npy', np.array(EPSILON_LIST))

After training the model, we want to visualize the results again. We use the z-score again and a plot how probable the secret is under the model in comparison to other potential secrets.

In [ ]:
# Plotting z-scores
x = range(1, epoch + 1)
plt.plot(x, Z_SCORE_LIST, label='z-score')
plt.xlabel('Epoch')
plt.ylabel('z-score')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.savefig("z_score_{}_dp.png".format(dataset['secret_format']).replace(' ', ''))
plt.show()
plt.close()
print(Z_SCORE_LIST)

# If we are using DP Optimization, we want to plot the epsilons, too
if dpsgd:
    plt.plot(x, EPSILON_LIST, label='epsilon')
    plt.xlabel('Epoch')
    plt.ylabel('epsilon')
    plt.legend()
    plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
    plt.savefig("epsilon_{}_dp.png".format(dataset['secret_format']).replace(' ', ''))
    plt.show()
    plt.close()
    print(EPSILON_LIST)

In [ ]:
x = np.linspace(-5, 5, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1), label='Normal distribution')
plt.scatter(Z_SCORE_LIST, stats.norm.pdf(Z_SCORE_LIST), marker='x', color='red', label='Secret Probabilities')
plt.xlabel('Standard deviations from mean')
plt.ylabel('Probability')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.show()

The results will show that the z-score is closer to the mean of the potential secrets and stays within the standard deviation. Note, that the log-perplexity of the secret is neither consistently low nor high, but rather more or less randomly distributed. This causes an attacker to be unable to reliably infer if the secret has been contained in the training data. 